- 1.      cctv 댓수, 최근 증가율(최근 3개월간 증가율), 인구수, 외국인 비율(%), 고령자 비율(%)
          - 구별이 인덱스
          - 계는 사라져야함

In [43]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

	인구	인구	인구	인구	인구	    인구	인구	    인구	    인구	세대당인구	65세이상고령자	합계	합계	합계	한국인	한국인	한국인	등록외국인	등록외국인	등록외국인	세대당인구	65세이상고령자
	계	    남자    여자	계	    남자	    여자	계	        남자	    여자	세대당인구	65세이상고령자

In [44]:
df = pd.read_csv('서울주민등록인구(구별)통계.txt', sep='\t', skiprows=2)
df.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2021.2/4,합계,"4,405,638","9,795,426","4,756,178","5,039,248","9,565,990","4,649,442","4,916,548","229,436","106,736","122,700",2.17,"1,576,971"
1,2021.2/4,종로구,"73,525","155,106","75,009","80,097","146,029","70,985","75,044","9,077","4,024","5,053",1.99,"27,605"
2,2021.2/4,중구,"62,897","132,259","64,526","67,733","123,016","60,019","62,997","9,243","4,507","4,736",1.96,"23,980"
3,2021.2/4,용산구,"111,650","240,665","116,858","123,807","226,378","108,948","117,430","14,287","7,910","6,377",2.03,"38,884"
4,2021.2/4,성동구,"134,542","295,767","143,927","151,840","289,162","141,042","148,120","6,605","2,885","3,720",2.15,"45,591"


In [45]:
# 추출할 컬럼 확인
df.columns

Index(['기간', '자치구', '세대', '계', '남자', '여자', '계.1', '남자.1', '여자.1', '계.2',
       '남자.2', '여자.2', '세대당인구', '65세이상고령자'],
      dtype='object')

In [46]:
# 귀찮으니 df에 계속 덮어쓰지만 틀리면 위에서 다시 실행해야함
df = df[['자치구','계','계.2','65세이상고령자']]
df.head(3)

,자치구,계,계.2,65세이상고령자
0,합계,"9,795,426","229,436","1,576,971"
1,종로구,"155,106","9,077","27,605"
2,중구,"132,259","9,243","23,980"


In [47]:
# 컬럼명 미리 바꿔줌
df.rename(columns={'계':'인구수', '계.2': '외국인','65세이상고령자':'고령자'}, inplace=True)
df.head(3)

,자치구,인구수,외국인,고령자
0,합계,"9,795,426","229,436","1,576,971"
1,종로구,"155,106","9,077","27,605"
2,중구,"132,259","9,243","23,980"


In [48]:
# 필요없는 컬럼 떨구고 인덱스 설정
df.drop([0], inplace=True)
df.set_index(['자치구'], inplace=True)
df.head(3)

,인구수,외국인,고령자
자치구,,,
종로구,"155,106","9,077","27,605"
중구,"132,259","9,243","23,980"
용산구,"240,665","14,287","38,884"


In [49]:
# 데이터 값 정수로 변환
for column in df.columns[:]:
    df[column] = df[column].apply(lambda x: int(x.replace(',','')))
df.head(3)

,인구수,외국인,고령자
자치구,,,
종로구,155106,9077,27605
중구,132259,9243,23980
용산구,240665,14287,38884


In [50]:
# 필요한 새로운 컬럼 만들기
df['외국인비율(%)'] = (df.외국인 / df.인구수 * 100).round(2)
df['고령자비율(%)'] = (df.고령자 / df.인구수 * 100).round(2)
population = df
population.head(3)


,인구수,외국인,고령자,외국인비율(%),고령자비율(%)
자치구,,,,,
종로구,155106,9077,27605,5.85,17.80
중구,132259,9243,23980,6.99,18.13
용산구,240665,14287,38884,5.94,16.16


### 서울시 CCTV 현황 데이터 추출 및 정제

In [51]:
df2 = pd.read_csv('서울시CCTV설치운영현황.csv', skiprows=1, encoding='euc-kr')
df2.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289
3,용산구,"2,383",34,71,234,125,221,298,351,125,307,617,0
4,성동구,"3,602",448,125,212,105,339,310,874,390,262,461,76


In [52]:
for column in df2.columns[1:]:
    df2[column] = df2[column].apply(lambda x: int(x.replace(',','')))

df2.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289


In [53]:
df2['구분'] = df2.구분.apply(lambda x: x.replace(' ',''))
# df2['구분'] = df2.구분.str.replace(' ','') 위와 같은 코드

In [54]:
df2.drop([0], inplace=True)
df2.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
2,중구,2333,16,114,87,77,236,240,372,386,155,361,289
3,용산구,2383,34,71,234,125,221,298,351,125,307,617,0
4,성동구,3602,448,125,212,105,339,310,874,390,262,461,76
5,광진구,2588,35,57,100,187,98,52,675,465,712,175,32


In [55]:
df2.set_index(['구분'], inplace=True)
df2.head()

,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
구분,,,,,,,,,,,,
종로구,1772,813,0,0,210,150,1,261,85,9,200,43
중구,2333,16,114,87,77,236,240,372,386,155,361,289
용산구,2383,34,71,234,125,221,298,351,125,307,617,0
성동구,3602,448,125,212,105,339,310,874,390,262,461,76
광진구,2588,35,57,100,187,98,52,675,465,712,175,32


In [56]:
df2['최근증가율(%)'] = ((df2.iloc[:,1:8].sum(axis=1)/df2.iloc[:,8:11].sum(axis=1))*100).round(2)
df2.tail()

,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,최근증가율(%)
구분,,,,,,,,,,,,,
관악구,4942,428,205,291,513,529,621,687,663,640,331,34,200.37
서초구,3704,116,75,94,71,563,516,1060,428,358,420,3,206.88
강남구,6502,124,77,75,597,840,1310,999,748,789,942,1,162.24
송파구,2854,72,61,86,85,215,148,241,542,1068,235,101,49.21
강동구,2547,332,27,163,49,154,202,273,377,356,614,0,89.09


In [57]:
df2.rename(columns={'총계':'CCTV댓수'}, inplace=True)
df2.index.names = ['구별']
cctv = df2[['CCTV댓수', '최근증가율(%)']]
cctv.head()

,CCTV댓수,최근증가율(%)
구별,,
종로구,1772,488.10
중구,2333,126.61
용산구,2383,127.17
성동구,3602,216.80
광진구,2588,89.05


In [58]:
# CCTV 댓수가 많은 자치구 Top 5
cctv.sort_values('CCTV댓수', ascending=False).head()

,CCTV댓수,최근증가율(%)
구별,,
강남구,6502,162.24
관악구,4942,200.37
구로구,4075,204.79
성북구,3958,105.79
은평구,3791,84.85


# 데이터 합칠 준비
    - 이너, 아우터(아우터면 먼 아우터할지)

In [59]:
cctv.index

Index(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구',
       '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구',
       '관악구', '서초구', '강남구', '송파구', '강동구'],
      dtype='object', name='구별')

In [60]:
population.index

Index(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구',
       '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구',
       '관악구', '서초구', '강남구', '송파구', '강동구'],
      dtype='object', name='자치구')

In [61]:
# 실수로 인덱스 이름 안 바꿈
population.index.names = ['구별']
population.head(1)

,인구수,외국인,고령자,외국인비율(%),고령자비율(%)
구별,,,,,
종로구,155106,9077,27605,5.85,17.8


In [62]:
# 그냥 합쳐도 될 듯
fusion = pd.merge(cctv, population, on="구별")
fusion

,CCTV댓수,최근증가율(%),인구수,외국인,고령자,외국인비율(%),고령자비율(%)
구별,,,,,,,
종로구,1772,488.10,155106,9077,27605,5.85,17.80
중구,2333,126.61,132259,9243,23980,6.99,18.13
용산구,2383,127.17,240665,14287,38884,5.94,16.16
성동구,3602,216.80,295767,6605,45591,2.23,15.41
광진구,2588,89.05,355306,12825,50815,3.61,14.30
동대문구,2497,286.96,352259,14219,61262,4.04,17.39
중랑구,3296,52.34,394679,4707,70172,1.19,17.78
성북구,3958,105.79,443206,9548,73568,2.15,16.60
강북구,2462,22.37,306093,3448,63383,1.13,20.71


In [63]:
# 데이터 내려받기
fusion.to_csv('서울시_데이터.csv')